# TOOL MCGVYERING: TOOL CREATION WITH AUXILARY OBJECTS


## Required Libraries & Parameters

In [1]:
import sys
sys.path.append('../src')
import numpy as np
import robotic as ry

from RAI import RAI
from SEG import SEG
from SQF import SQF
from Score import Score

cam_list = ["cam_front", "cam_back", "cam_left", "cam_right", "cam_up", "cam_down"]
filter = 1
model_path = "../src/models/tools/simple/parts"
target_path = "../src/point_clouds_ref/"
scene = ry.Config()
scene.addFile("../src/config/tools_simple1_s.g")
scene.view(False)

reference_tool = "shovel" # Select a reference tool

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## STEP 1: Generate the point cloud for the reference tool

In [2]:
print("Generating the reference point cloud")
rai = RAI(verbose=0)
segments_ref = rai.generate_ref_point_cloud(model_path, cam_list, filter = filter, object_name=reference_tool)

Generating the reference point cloud
Model name: shovel_head_s
Model name: shovel_handle_s


## STEP 2: Generate the point cloud for the candidate objects

In [3]:
print("Generating the candidate point cloud")
cam_list = ["cam_front_left", "cam_front_right", "cam_back_right", "cam_back_left", "cam_up", "cam_front", "cam_left", "cam_back"] 
ptc_cand, _, _ = rai.get_raw_point_cloud(scene, cam_list, filter = filter)
   

Generating the candidate point cloud


## STEP 3: Remove the plane from the scene

In [4]:
print("Removing the plane from the scene")
seg = SEG(verbose=0)
pcl_filtered = seg.RANSAC_plane(ptc_cand)
segments_cand = seg.segment_objects(pcl_filtered, scene)

Removing the plane from the scene


## STEP 4: Fit SuperQuadric function to the objects

In [5]:
print("Fitting SuperQuadric function to the objects")
param_ref  = {}
for key, pcd in segments_ref.items():
    sqf = SQF(pcd, verbose=0)
    param, _, _, _ = sqf.fit(sq_type=[0])
    if len(param) < 12:
        param = np.pad(param, (0, 12 - len(param)), mode='constant', constant_values=0)
    param_ref[key] = param

param_cand = {}
for key, pcd in segments_cand.items():
    sqf = SQF(pcd, verbose=0)
    param, _, _, _ = sqf.fit(sq_type=[0])
    if len(param) < 12:
        param = np.pad(param, (0, 12 - len(param)), mode='constant', constant_values=0)
    param_cand[key] = param

Fitting SuperQuadric function to the objects
Both `ftol` and `xtol` termination conditions are satisfied.
Function evaluations 231, initial cost 7.9156e+00, final cost 8.7928e-01, first-order optimality 1.05e-07.
`xtol` termination condition is satisfied.
Function evaluations 1653, initial cost 5.4748e-01, final cost 7.1232e-05, first-order optimality 3.32e-07.
`gtol` termination condition is satisfied.
Function evaluations 1634, initial cost 2.1509e-01, final cost 6.8433e-05, first-order optimality 9.97e-09.
`ftol` termination condition is satisfied.
Function evaluations 400, initial cost 2.3696e+02, final cost 5.3247e-03, first-order optimality 3.55e-06.
`ftol` termination condition is satisfied.
Function evaluations 275, initial cost 1.1473e+00, final cost 8.3112e-02, first-order optimality 2.90e-05.


## STEP 5: Find the best matching candidates objects to the reference tool set

In [6]:
print("Finding the best matching candidate objects to the reference tool set")
score = Score(param_ref, param_cand, verbose=1)
match_1, match_2 = score.algorithm_1()

Finding the best matching candidate objects to the reference tool set
shovel_head_s object matches with obj_hammer_head object
shovel_handle_s object matches with obj_axe_handle object


## STEP 6: Create the tool

In [7]:
rai.create_tool(scene, match_1, match_2)

-- pairCollision.cpp:libccd:354(0) WARNING: called MPR penetration for non intersecting meshes...
-- pairCollision.cpp:libccd:359(0) WARNING: but GJK says intersection
-- pairCollision.cpp:libccd:354(0) WARNING: called MPR penetration for non intersecting meshes...
-- pairCollision.cpp:libccd:359(0) WARNING: but GJK says intersection
-- pairCollision.cpp:libccd:354(0) WARNING: called MPR penetration for non intersecting meshes...
-- pairCollision.cpp:libccd:359(0) WARNING: but GJK says intersection
-- pairCollision.cpp:libccd:354(0) WARNING: called MPR penetration for non intersecting meshes...
-- pairCollision.cpp:libccd:359(0) WARNING: but GJK says intersection
-- pairCollision.cpp:libccd:354(0) WARNING: called MPR penetration for non intersecting meshes...
-- pairCollision.cpp:libccd:359(0) WARNING: but GJK says intersection
-- pairCollision.cpp:libccd:354(0) WARNING: called MPR penetration for non intersecting meshes...
-- pairCollision.cpp:libccd:359(0) WARNING: but GJK says inter

False